In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import numpy  as np
import seaborn as sns
%matplotlib inline

path = './data/'
pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

## 데이터로드

In [2]:
sigungu = pd.read_csv(path+'sigungu_imp_10.csv', encoding='cp949')
sigungu_3 = pd.read_csv(path+'시군구별지역안전지표_2015.csv', encoding='cp949')

In [3]:
y_bin = sigungu.iloc[:,-7:]    #등급(binary)
y_deci = sigungu_3.iloc[:,-7:] #등급(1-5)
X = sigungu.iloc[:,4:-7]       #독립변수X
name = sigungu.iloc[:,:4]      #이름

In [4]:
y_bin.columns = ['fire','transport','disaster','crime','accident','suicide','infection']

In [5]:
sigungu_xy = pd.concat([X,y_bin], axis=1)
sigungu_nxy = pd.concat([name, sigungu_xy,y_deci], axis=1)
sigungu_nxy.shape

(226, 162)

In [6]:
bokgi = pd.read_csv(path+'15_복지예산_시군구.csv', encoding='cp949')
bokgi

,시도,시군구,사회복지비율,Unnamed: 3,Unnamed: 4
0,서울특별시,전국계,27.46,NaN,NaN
1,서울특별시,계,34.13,NaN,NaN
2,서울특별시,본청,33.57,NaN,NaN
3,서울특별시,구계,51.46,NaN,NaN
4,서울특별시,종로구,34.99,NaN,NaN
...,...,...,...,...,...
281,경상남도,함양군,20.18,NaN,NaN
282,경상남도,거창군,22.03,NaN,NaN
283,경상남도,합천군,19.44,NaN,NaN
284,NaN,NaN,NaN,NaN,NaN


###  2016년 사망자수, 사망률

In [7]:
deth = pd.read_csv(path+'2016년_사망자수_사망률.csv', encoding='cp949')
deth.head(1)

,시도,시군구,자살수2016,자살률2016
0,서울특별시,서울특별시,2261,23


### 노인비율위한것(고령화지수)

In [8]:
go_jisu = pd.read_csv(path+'노인비율위한것.csv', encoding='cp949')
go_jisu.head()

,Unnamed: 0,시,시군구,연령,총인구,시도
0,1,서울특별시,서울특별시,0~4세,391765,서울특별시
1,2,서울특별시,서울특별시,5~9세,373253,서울특별시
2,3,서울특별시,서울특별시,10~14세,404973,서울특별시
3,4,서울특별시,서울특별시,65세이상,1221006,서울특별시
4,5,서울특별시,종로구,0~4세,4485,서울특별시


In [9]:
old_jisu=go_jisu.pivot_table(values='총인구', index=['시도','시군구'], columns='연령')
old_jisu.head(1)

,연령,0~4세,10~14세,5~9세,65세이상
시도,시군구,,,,
강원도,강릉시,7362,10229,8368,36202


In [10]:
old_jisu['0~14세'] = old_jisu['0~4세']+old_jisu['10~14세']
old_jisu['고령화지수'] =old_jisu['65세이상']/old_jisu['0~14세']*100

In [11]:
# old_jisu.index = old_jisu.index.set_names(['시도','시군구'])
old_jisu = old_jisu.reset_index(level=['시도','시군구'],col_level=1)
old_jisu

연령,시도,시군구,0~4세,10~14세,5~9세,65세이상,0~14세,고령화지수
0,강원도,강릉시,7362,10229,8368,36202,17591,205.798420
1,강원도,강원도,58045,72393,63331,254803,130438,195.344148
2,강원도,고성군,735,940,721,6641,1675,396.477612
3,강원도,동해시,3827,4794,4246,14575,8621,169.063914
4,강원도,삼척시,2251,2724,2412,14019,4975,281.788945
...,...,...,...,...,...,...,...,...
273,충청북도,청원구,11252,9092,10115,19359,20344,95.158278
274,충청북도,청주시,42386,44123,41810,87881,86509,101.585962
275,충청북도,충주시,7735,9879,8534,33507,17614,190.229363
276,충청북도,충청북도,69659,76851,70899,230533,146510,157.349669


In [12]:
old_jisu.loc[:,['시도','시군구','고령화지수']]

연령,시도,시군구,고령화지수
0,강원도,강릉시,205.798420
1,강원도,강원도,195.344148
2,강원도,고성군,396.477612
3,강원도,동해시,169.063914
4,강원도,삼척시,281.788945
...,...,...,...
273,충청북도,청원구,95.158278
274,충청북도,청주시,101.585962
275,충청북도,충주시,190.229363
276,충청북도,충청북도,157.349669


### 노인비율위한 데이터(60세이상/전체인구)

In [13]:
old_ratio = pd.read_csv(path+'2015년 노인비율위한 데이터.csv', encoding='cp949')
old_ratio.head(1)

,시군구,시도,계,60세,65세,70세,75세,80세,85세,90세,95세,100세
0,서울특별시,서울특별시,10022181,585240,452512,347368,234999,130345,62456,23212,6507,5037


In [14]:
old_ratio['노인비율'] = ((old_ratio['60세']+old_ratio['65세']+old_ratio['70세']+old_ratio['75세']+\
                     old_ratio['80세']+old_ratio['85세']+old_ratio['90세']+old_ratio['95세']+old_ratio['100세'])/old_ratio['계'])*100

In [15]:
old_ratio.head(1)

,시군구,시도,계,60세,65세,70세,75세,80세,85세,90세,95세,100세,노인비율
0,서울특별시,서울특별시,10022181,585240,452512,347368,234999,130345,62456,23212,6507,5037,18.435867


## 지표추가

In [16]:
#의료시설_접근가능_시설수
hop_acc = pd.read_csv(path+'의료시설_접근가능_시설수_수정(15,30).csv', encoding='cp949')
hop_acc.rename(columns={'행정구역별(1)': '시도','행정구역별(2)':'시군구'}, inplace=True)
hop_acc.shape
hop_acc['공공의료'] = hop_acc['공공의료시설(15분)']+hop_acc['공공의료시설(30분)']+hop_acc['종합병원(15분)']+hop_acc['종합병원(30분)']

In [17]:
hop_acc.head(1)

,시도,시군구,시간대별(1),공공의료시설(15분),공공의료시설(30분),병_의원(15분),병_의원(30분),종합병원(15분),종합병원(30분),공공의료
0,전국,소계,일평균(06-20시),0.45,3.08,10.0,10.0,0.59,4.16,8.28


### 의료시설_유형별_평균접근시간 처리

In [18]:
#의료시설_유형별_평균접근시간.csv
hop_time = pd.read_csv(path+'의료시설_유형별_평균접근시간.csv', encoding='cp949')
hop_time
hop_time.rename(columns={'행정구역별(1)': '시도','행정구역별(2)':'시군구','시간대별(1)':'시간','공공의료시설':'공공의료'},inplace=True)
hop_time.head(1)

,시도,시군구,시간,공공의료,종합병원
0,전국,소계,저녁첨두(18-20시),18.85,27.84


In [19]:
#의료시설_유형별_접근시간 시간대 구분 --  저녁, 일평균
hop_d_time = hop_time.loc[hop_time['시간']=='일평균(06-20시)',:]
hop_d_time['일접근평균시간_병원'] = (hop_d_time['공공의료']+hop_d_time['종합병원'])/2
hop_d_time.head(1)

C:\Users\COM\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,시도,시군구,시간,공공의료,종합병원,일접근평균시간_병원
1,서울특별시,소계,일평균(06-20시),16.46,14.69,15.575


### 광역교통시설_유형별 데이터 처리

In [20]:
wide_acc = pd.read_csv(path+'광역교통시설_유형별_평균접근시간.csv', encoding='cp949')
wide_acc.head(1)

,시도,시군구,시간대,버스터미널,철도역
0,서울특별시,소계,일평균(06-20시),30.63,24.32


#### 광역교통시설_버스,철도_평균

In [21]:
wide_d_acc = wide_acc.loc[(hop_time['시간']=='일평균(06-20시)'),:]
wide_d_acc['일접근평균시간_광역'] = (wide_d_acc['버스터미널']+wide_d_acc['철도역'])/2
wide_d_acc.head(1)

C:\Users\COM\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,시도,시군구,시간대,버스터미널,철도역,일접근평균시간_광역
1,서울특별시,소계,저녁첨두(18-20시),30.51,24.04,27.275


In [22]:
wide_n_acc = wide_acc.loc[~(hop_time['시간']=='일평균(06-20시)'),:]
wide_n_acc['야간접근평균시간'] = (wide_n_acc['버스터미널']+wide_n_acc['철도역'])/2
wide_n_acc.head(1)

C:\Users\COM\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,시도,시군구,시간대,버스터미널,철도역,야간접근평균시간
0,서울특별시,소계,일평균(06-20시),30.63,24.32,27.475


### 판매시설(대형마트)_접근시간 처리

In [23]:
store_acc = pd.read_csv(path+'판매시설_유형별_평균접근시간.csv', encoding='cp949')
store_acc.head(1)

,시도,시군구,시간,대중교통/도보
0,서울특별시,소계,일평균(06-20시),12.05


In [24]:
store_d_acc = store_acc.loc[(store_acc['시간']=='일평균(06-20시)'),:]
store_n_acc = store_acc.loc[~(store_acc['시간']=='일평균(06-20시)'),:]
store_d_acc.head(1)

,시도,시군구,시간,대중교통/도보
0,서울특별시,소계,일평균(06-20시),12.05


### 원본 데이터에서 쓸 열들만 정리

In [25]:
##원본 데이터에서 쓸 열들만 정리
sigungu_1 = sigungu_nxy.loc[:,['시도','시군구','지역','1인가구수','건강보험급여실적',
                               '기초수급자수','범죄발생건수(살인)','주민등록인구(14세이하)',
                               '자살 사망자수','주민등록인구','주민등록인구(60세이상)',
                               '독거노인수','반지하가구 수','우울감경험률','suicide',
                               '자살','재정자주도','재정자립도','고령인구수',
                              '의료보험료','범죄발생건수(폭력)','기초수급자수(65세이상)',
                              '감염병 사망자수','시군구내 전입자수','장애인수',
                              '질병이환 및 사망외인으로 인한 사망자수','초등학생수',
                               '구급발생건수','독거노인수','순이동자수','시가화율']]
sigungu_1.shape

(226, 31)

### 풀 데이터셋 만들기

In [26]:
sigungu_ht=sigungu_1.merge(hop_d_time.loc[:,['시도','시군구','일접근평균시간_병원']] , on=['시도','시군구']).merge(store_d_acc.loc[:,['시도','시군구','대중교통/도보']], on=['시도','시군구'])\
.merge(wide_d_acc.loc[:,['시도','시군구','일접근평균시간_광역']], on=['시도','시군구']).merge(hop_acc.loc[:,['시도','시군구','공공의료']], on=['시도','시군구'])
sigungu_ht.head(1)

,시도,시군구,지역,1인가구수,건강보험급여실적,기초수급자수,범죄발생건수(살인),주민등록인구(14세이하),자살 사망자수,주민등록인구,주민등록인구(60세이상),독거노인수,반지하가구 수,우울감경험률,suicide,자살,재정자주도,재정자립도,고령인구수,의료보험료,범죄발생건수(폭력),기초수급자수(65세이상),감염병 사망자수,시군구내 전입자수,장애인수,질병이환 및 사망외인으로 인한 사망자수,초등학생수,구급발생건수,독거노인수,순이동자수,시가화율,일접근평균시간_병원,대중교통/도보,일접근평균시간_광역,공공의료
0,강원도,강릉시,2015_강원도_강릉시,1161.45445,1559476.199,314.30862,0.13901,1283.55429,3.66068,215807.0,2266.60859,404.29643,22.52012,4.7,1,4,64.61,17.39,1668.57424,6229719.75,68.02374,108.15219,2.03886,773.56156,605.49472,7.08967,522.87461,32.34371,404.29643,-37.53354,2.25,26.71,29.96,31.135,3.46


#### 고령화지수 매개변수 넣기

In [27]:
sigungu_ht = sigungu_ht.merge(old_jisu.loc[:,['시도','시군구','고령화지수']], on=['시도','시군구'])
sigungu_ht.shape

(226, 36)

#### 노인비율 매개변수 넣기

In [28]:
sigungu_ht = sigungu_ht.merge(old_ratio.loc[:,['시도','시군구','노인비율']], on=['시도','시군구'])
sigungu_ht.shape

(226, 37)

In [29]:
sigungu_ht['평균접근시간'] = (sigungu_ht['일접근평균시간_병원']+sigungu_ht['대중교통/도보']+sigungu_ht['일접근평균시간_광역'])/3

#### 2016년 사망,사망률 넣기

In [30]:
sigungu_ht =sigungu_ht.merge(deth, on=['시도','시군구'])
sigungu_ht.shape

(226, 40)

In [31]:
bokgi.iloc[:,:-2]

,시도,시군구,사회복지비율
0,서울특별시,전국계,27.46
1,서울특별시,계,34.13
2,서울특별시,본청,33.57
3,서울특별시,구계,51.46
4,서울특별시,종로구,34.99
...,...,...,...
281,경상남도,함양군,20.18
282,경상남도,거창군,22.03
283,경상남도,합천군,19.44
284,NaN,NaN,NaN


In [32]:
sigungu_ht = sigungu_ht.merge(bokgi.iloc[:,:-2], on=['시도','시군구'])

In [33]:
sigungu_ht.head(1)

,시도,시군구,지역,1인가구수,건강보험급여실적,기초수급자수,범죄발생건수(살인),주민등록인구(14세이하),자살 사망자수,주민등록인구,주민등록인구(60세이상),독거노인수,반지하가구 수,우울감경험률,suicide,자살,재정자주도,재정자립도,고령인구수,의료보험료,범죄발생건수(폭력),기초수급자수(65세이상),감염병 사망자수,시군구내 전입자수,장애인수,질병이환 및 사망외인으로 인한 사망자수,초등학생수,구급발생건수,독거노인수,순이동자수,시가화율,일접근평균시간_병원,대중교통/도보,일접근평균시간_광역,공공의료,고령화지수,노인비율,평균접근시간,자살수2016,자살률2016,사회복지비율
0,강원도,강릉시,2015_강원도_강릉시,1161.45445,1559476.199,314.30862,0.13901,1283.55429,3.66068,215807.0,2266.60859,404.29643,22.52012,4.7,1,4,64.61,17.39,1668.57424,6229719.75,68.02374,108.15219,2.03886,773.56156,605.49472,7.08967,522.87461,32.34371,404.29643,-37.53354,2.25,26.71,29.96,31.135,3.46,205.79842,23.836689,29.268333,73,34.4,31.57


### 회귀식 만들어서 주요 변수 탐색

In [35]:
import statsmodels.api as sm
sigungu_ht1 = sigungu_ht
sigungu_ht1.rename(columns={'반지하가구 수':'반지하가구수',
                            '1인가구수':'일인가구수',
                            '범죄발생건수(살인)':'살인범죄건수',
                            '자살 사망자수':'자살사망자수',
                            '주민등록인구(14세이하)':'십사세이하주민등록인구',
                            '주민등록인구(60세이상)':'육십세이상주민등록인구',
                            '기초수급자수(65세이상)':'육십오세이상기초수급자수',
                            '범죄발생건수(폭력)':'폭력범죄발생건수',
                            '감염병 사망자수':'감염병사망자수',
                            '시군구내 전입자수':'시군구내전입자수',
                            '질병이환 및 사망외인으로 인한 사망자수':'질병이환및사망자외인으로인한사망자수',
                            '반지하가구 수':'반지하가구수'
                                }, inplace=True)
reg = sm.OLS.from_formula("자살 ~ 일인가구수+\
                          기초수급자수+\
                          자살사망자수+십사세이하주민등록인구+\
                          의료보험료+\
                          시군구내전입자수+주민등록인구+\
                          평균접근시간+\
                          반지하가구수+재정자립도+사회복지비율\
                          ", sigungu_ht1).fit()
reg.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     자살   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     86.36
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           3.55e-72
Time:                        15:36:42   Log-Likelihood:                -157.61
No. Observations:                 225   AIC:                             339.2
Df Residuals:                     213   BIC:                             380.2
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.4882      0.631      0.774      0.440      -0.755       1.732
일인가구수           0.0006      0.000      2.965      0.003       0.000       0.001
기초수급자수          0.0022      0.000      5.210      0.000       0.001       0.003
자살사망자수          0.8093      0.040     20.143      0.000       0.730       0.889
십사세이하주민등록인구    -0.0011      0.000     -4.767      0.000      -0.001      -0.001
의료보험료       -1.318e-07   3.35e-08     -3.933      0.000   -1.98e-07   -6.57e-08
시군구내전입자수        0.0014      0.000      5.991      0.000       0.001       0.002
주민등록인구      -1.063e-06   2.68e-07     -3.960      0.000   -1.59e-06   -5.34e-07
평균접근시간         -0.0102      0.003     -3.348      0.001      -0.016      -0.004
반지하가구수       7.301e-05      0.000      0.171      0.865      -0.001       0.001
재정자립도           0.0299      0.006      5.054      0.000       0.018       0.041
사회복지비율          0.0125      0.005      2.623      0.009       0.003       0.022
==============================================================================
Omnibus:                        3.200   Durbin-Watson:                   1.826
Prob(Omnibus):                  0.202   Jarque-Bera (JB):                3.018
Skew:                           0.172   Prob(JB):                        0.221
Kurtosis:                       3.452   Cond. No.                     1.35e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.35e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""